In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date, time, datetime

In [2]:
train_users = pd.read_csv(r"C:\Users\Ben\Desktop\Project3_DS_310\train_users.csv")
test_users = pd.read_csv(r"C:\Users\Ben\Desktop\Project3_DS_310\test_users.csv")
sessions = pd.read_csv(r"C:\Users\Ben\Desktop\Project3_DS_310\sessions.csv")

In [3]:
train_users.drop(['country_destination'], axis = 1)
#Concatenating train and test data for EDA
df_all = pd.concat((train_users, test_users), axis = 0, ignore_index= True)

In [4]:
train_users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [5]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
missing_values_table(train_users)

Your selected dataframe has 16 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
date_first_booking,124543,58.3
age,87990,41.2
first_affiliate_tracked,6065,2.8


In [7]:

dates_list = []
year_list = []
month_list = []

for i in df_all["date_account_created"]:
    date_time_obj = datetime.strptime(i, '%Y-%m-%d')
    year = date_time_obj.date().year
    month = date_time_obj.date().month
    dates_list.append(date_time_obj)
    year_list.append(year)
    month_list.append(month)

df_all["datetime_account_created"] = dates_list
df_all["year_account_created"] = year_list
df_all["month_account_created"] = month_list

In [8]:
dates_list = []
year_list = []
month_list = []

for i in df_all["timestamp_first_active"]:
    time_stamp_obj = pd.to_datetime(i , format='%Y%m%d%H%M%S')
    year = time_stamp_obj.date().year
    month = time_stamp_obj.date().month
    dates_list.append(time_stamp_obj)
    year_list.append(year)
    month_list.append(month)
    
df_all["datetime_first_active"] = dates_list
df_all["year_first_active"] = year_list
df_all["month_first_active"] = month_list

In [9]:
df_all["active_after_created"] = df_all["datetime_first_active"] - df_all["datetime_account_created"]

In [10]:
df_all['age'].quantile(0.5)

33.0

In [11]:
df_all['age'] = np.where(df_all['age'] > 110, 34, df_all['age'])
df_all['age'].describe()

count    158681.000000
mean         36.838115
std          13.766690
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max         110.000000
Name: age, dtype: float64

In [12]:
by_signup_device = df_all.groupby(['signup_method', 'first_device_type'])

In [13]:
df_all['first_affiliate_tracked'].value_counts()

untracked        143181
linked            62064
omg               54859
tracked-other      6655
product            2353
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64

In [14]:
tracked = []

for i in df_all['first_affiliate_tracked']:
    if i == "untracked":
        isTracked = 1
    else:
        isTracked = 0
    tracked.append(isTracked)

df_all['is_first_affiliate_tracked'] = tracked

In [15]:
df_all.isnull().sum()

id                                 0
date_account_created               0
timestamp_first_active             0
date_first_booking            186639
gender                             0
age                           116866
signup_method                      0
signup_flow                        0
language                           0
affiliate_channel                  0
affiliate_provider                 0
first_affiliate_tracked         6085
signup_app                         0
first_device_type                  0
first_browser                      0
country_destination            62096
datetime_account_created           0
year_account_created               0
month_account_created              0
datetime_first_active              0
year_first_active                  0
month_first_active                 0
active_after_created               0
is_first_affiliate_tracked         0
dtype: int64

In [16]:
missing_values_table(df_all)

Your selected dataframe has 24 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
date_first_booking,186639,67.7
age,116866,42.4
country_destination,62096,22.5
first_affiliate_tracked,6085,2.2


In [17]:
signup = df_all[["signup_app"]]

In [18]:
signupdf = pd.get_dummies(signup)

In [19]:
signupdf

,signup_app_Android,signup_app_Moweb,signup_app_Web,signup_app_iOS
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
275542,0,0,1,0
275543,1,0,0,0
275544,0,0,1,0
275545,0,0,1,0


In [20]:
train = train_users 
test = test_users 

In [21]:
train

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,NaN,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF
213447,mhewnxesx9,2014-06-30,20140630235719,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF
213448,6o3arsjbb4,2014-06-30,20140630235754,NaN,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
213449,jh95kwisub,2014-06-30,20140630235822,NaN,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF


In [22]:
df_all

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_browser,country_destination,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,active_after_created,is_first_affiliate_tracked
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,...,Chrome,NDF,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,-466 days +04:32:55,1
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,...,Chrome,NDF,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,-732 days +17:48:09,1
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,IE,US,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,-476 days +23:12:47,1
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,Firefox,other,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,-765 days +06:01:29,1
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,Chrome,US,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,-280 days +06:11:05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31.0,basic,0,en,direct,...,IE,NaN,2014-09-30,2014,9,2014-09-30 23:52:32,2014,9,0 days 23:52:32,1
275543,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,NaN,basic,23,ko,direct,...,-unknown-,NaN,2014-09-30,2014,9,2014-09-30 23:53:06,2014,9,0 days 23:53:06,1
275544,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,NaN,basic,0,de,direct,...,Firefox,NaN,2014-09-30,2014,9,2014-09-30 23:54:08,2014,9,0 days 23:54:08,0
275545,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,NaN,basic,0,en,sem-brand,...,Safari,NaN,2014-09-30,2014,9,2014-09-30 23:54:30,2014,9,0 days 23:54:30,0


In [23]:
#Non owen code to copy
method = pd.get_dummies(df_all[["signup_method"]])
affch = pd.get_dummies(df_all[["affiliate_channel"]])
affprov = pd.get_dummies(df_all[["affiliate_provider"]])
firstdevice = pd.get_dummies(df_all[["first_device_type"]])
browser = pd.get_dummies(df_all[["first_browser"]])
signup = pd.get_dummies(df_all[["signup_app"]])
genderdum = pd.get_dummies(df_all[["gender"]])



In [24]:
browser

,first_browser_-unknown-,first_browser_AOL Explorer,first_browser_Android Browser,first_browser_Apple Mail,first_browser_Arora,first_browser_Avant Browser,first_browser_BlackBerry Browser,first_browser_Camino,first_browser_Chrome,first_browser_Chrome Mobile,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275543,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df2 = df_all

In [35]:
df2 = pd.concat([df2.reset_index(drop=True),method.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affch.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affprov.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),firstdevice.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),browser.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),genderdum.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),signup.reset_index(drop=True)], axis=1)

In [26]:
#df2 = pd.concat([df_all.reset_index(drop=True),signup.reset_index(drop=True)], axis=1)

In [36]:
df2

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_browser_Yandex.Browser,first_browser_wOSBrowser,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_app_Android,signup_app_Moweb,signup_app_Web,signup_app_iOS
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,...,0,0,1,0,0,0,0,0,1,0
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,...,0,0,0,0,1,0,0,0,1,0
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,0,0,0,1,0,0,0,0,1,0
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,0,0,0,1,0,0,0,0,1,0
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31.0,basic,0,en,direct,...,0,0,1,0,0,0,0,0,1,0
275543,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,NaN,basic,23,ko,direct,...,0,0,1,0,0,0,1,0,0,0
275544,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,NaN,basic,0,de,direct,...,0,0,1,0,0,0,0,0,1,0
275545,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,NaN,basic,0,en,sem-brand,...,0,0,1,0,0,0,0,0,1,0


In [37]:
missing_values_table(df2)

Your selected dataframe has 130 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
date_first_booking,186639,67.7
age,116866,42.4
country_destination,62096,22.5
first_affiliate_tracked,6085,2.2


In [27]:
'''
method = pd.get_dummies([["signup_method"]])
affch = pd.get_dummies(df_all[["affiliate_channel"]])
affprov = pd.get_dummies(df_all[["affiliate_provider"]])
firstdevice = pd.get_dummies(df_all[["first_device_type"]])
browser = pd.get_dummies(df_all[["first_browser"]])
'''

'\nmethod = pd.get_dummies([["signup_method"]])\naffch = pd.get_dummies(df_all[["affiliate_channel"]])\naffprov = pd.get_dummies(df_all[["affiliate_provider"]])\nfirstdevice = pd.get_dummies(df_all[["first_device_type"]])\nbrowser = pd.get_dummies(df_all[["first_browser"]])\n'

In [28]:
method

,signup_method_basic,signup_method_facebook,signup_method_google,signup_method_weibo
0,0,1,0,0
1,0,1,0,0
2,1,0,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
275542,1,0,0,0
275543,1,0,0,0
275544,1,0,0,0
275545,1,0,0,0


In [29]:
affch

,affiliate_channel_api,affiliate_channel_content,affiliate_channel_direct,affiliate_channel_other,affiliate_channel_remarketing,affiliate_channel_sem-brand,affiliate_channel_sem-non-brand,affiliate_channel_seo
0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
275542,0,0,1,0,0,0,0,0
275543,0,0,1,0,0,0,0,0
275544,0,0,1,0,0,0,0,0
275545,0,0,0,0,0,1,0,0


In [30]:
affprov

,affiliate_provider_baidu,affiliate_provider_bing,affiliate_provider_craigslist,affiliate_provider_daum,affiliate_provider_direct,affiliate_provider_email-marketing,affiliate_provider_facebook,affiliate_provider_facebook-open-graph,affiliate_provider_google,affiliate_provider_gsp,affiliate_provider_meetup,affiliate_provider_naver,affiliate_provider_other,affiliate_provider_padmapper,affiliate_provider_vast,affiliate_provider_wayn,affiliate_provider_yahoo,affiliate_provider_yandex
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
275543,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
275544,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
275545,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [31]:
firstdevice

,first_device_type_Android Phone,first_device_type_Android Tablet,first_device_type_Desktop (Other),first_device_type_Mac Desktop,first_device_type_Other/Unknown,first_device_type_SmartPhone (Other),first_device_type_Windows Desktop,first_device_type_iPad,first_device_type_iPhone
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
275542,0,0,0,0,0,0,1,0,0
275543,1,0,0,0,0,0,0,0,0
275544,0,0,0,0,0,0,1,0,0
275545,0,0,0,1,0,0,0,0,0


In [32]:
browser

,first_browser_-unknown-,first_browser_AOL Explorer,first_browser_Android Browser,first_browser_Apple Mail,first_browser_Arora,first_browser_Avant Browser,first_browser_BlackBerry Browser,first_browser_Camino,first_browser_Chrome,first_browser_Chrome Mobile,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275543,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train = df2.iloc[0:len(train),:]
test = df2.iloc[len(train):,:]

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 79 columns):
 #   Column                              Non-Null Count   Dtype          
---  ------                              --------------   -----          
 0   id                                  213451 non-null  object         
 1   date_account_created                213451 non-null  object         
 2   timestamp_first_active              213451 non-null  int64          
 3   date_first_booking                  88908 non-null   object         
 4   gender                              213451 non-null  object         
 5   age                                 125461 non-null  float64        
 6   signup_method                       213451 non-null  object         
 7   signup_flow                         213451 non-null  int64          
 8   language                            213451 non-null  object         
 9   affiliate_channel                   213451 non-null  object         
 

In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62096 entries, 213451 to 275546
Data columns (total 79 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   id                                  62096 non-null  object         
 1   date_account_created                62096 non-null  object         
 2   timestamp_first_active              62096 non-null  int64          
 3   date_first_booking                  0 non-null      object         
 4   gender                              62096 non-null  object         
 5   age                                 33220 non-null  float64        
 6   signup_method                       62096 non-null  object         
 7   signup_flow                         62096 non-null  int64          
 8   language                            62096 non-null  object         
 9   affiliate_channel                   62096 non-null  object         
 10  affi